# PSIthon, PsiAPI, and QCSchema:
### Data Paths to Use Psi4

##### 4 December 2020
##### Lori A. Burns

In [1]:
# conda env: qca2
# This presentation mode for Jupyter: `conda install rise -c conda-forge`
# in Edit Notebook Metadata, `"rise": {"scroll": true}`

import pprint
import json

from IPython.display import IFrame

import qcelemental as qcel
from qcelemental.models import Molecule
print(qcel.__version__)

import qcengine as qcng
print(qcng.__version__)  

import psi4
print(psi4.__version__)

v0.16.0
v0.16.0
1.4a2.dev1023


In [2]:
IFrame("./threeinputs.png", width=931, height=353)

## Psithon

* special character-reducing syntax for `molecule{...}`, `set`, `basisset{...}`
* `psi4` pre-imported so can use `energy()` and `print_variables()` without namespace
* runs through a preprocessor that converts the special syntax to proper Python
* domain-specific language
* only input mode before 2016 C++/Python "inversion"
* run as:
  * `> psi4 psithon.in`

In [3]:
with open("psithon.in", "w") as fp:
    fp.write("""molecule {
Ne
Ne 1 3.0
}
 
set freeze_core True

energy("ccsd(t)/cc-pvtz")
""")

In [4]:
!psi4 psithon.in
with open("psithon.out") as fp:
    print(fp.read())

hi

    -----------------------------------------------------------------------
          Psi4: An Open-Source Ab Initio Electronic Structure Package
                               Psi4 1.4a2.dev1023 

                         Git: Rev {master} 7d35d14 


    D. G. A. Smith, L. A. Burns, A. C. Simmonett, R. M. Parrish,
    M. C. Schieber, R. Galvelis, P. Kraus, H. Kruse, R. Di Remigio,
    A. Alenaizan, A. M. James, S. Lehtola, J. P. Misiewicz, M. Scheurer,
    R. A. Shaw, J. B. Schriber, Y. Xie, Z. L. Glick, D. A. Sirianni,
    J. S. O'Brien, J. M. Waldrop, A. Kumar, E. G. Hohenstein,
    B. P. Pritchard, B. R. Brooks, H. F. Schaefer III, A. Yu. Sokolov,
    K. Patkowski, A. E. DePrince III, U. Bozkaya, R. A. King,
    F. A. Evangelista, J. M. Turney, T. D. Crawford, C. D. Sherrill,
    J. Chem. Phys. 152(18) 184108 (2020). https://doi.org/10.1063/5.0006002

                            Additional Code Authors
    E. T. Seidl, C. L. Janssen, E. F. Valeev, M. L. Leininger,
    J. F. Gon

## PsiAPI
* pure Python API
* need to mind strings, bools, floats
* commands need `psi4.`
* some convenience functions replace Psithon's special syntax
  * `molecule name {...}` --> `name = psi4.geometry("""...""")`
  * `set keyword value` --> `psi4.set_options({"keyword": value})`
  * `set module keyword value` --> `psi4.set_options({"module__keyword": value})
* fundamental input mode that all others use
* run as:
  * `> python psiapi.py`
  * `> psi4 psiapi.py`
  * in python session, `import psi4`, `psi4.geometry(...`, `psi4.energy(...`, etc.

In [5]:
with open("psiapi.py", "w") as fp:
    fp.write('''import psi4

psi4.geometry("""
Ne
Ne 1 3.0
""")

psi4.set_options({
 "freeze_core": "True"})

psi4.energy("ccsd(t)/cc-pvtz")
''')

In [6]:
!python psiapi.py


Scratch directory: /tmp/

Scratch directory: /tmp/

*** tstart() called on ariadne
*** at Mon Dec  7 14:52:26 2020

   => Loading Basis Set <=

    Name: CC-PVTZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1-2 entry NE         line   338 file /Users/loriab/linux/miniconda3/envs/qca2/share/psi4/basis/cc-pvtz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: d2h
    Full point group: D_inf_h

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  ---------

	Sorting half-transformed integrals.
	First half integral transformation complete.
	Starting second half-transformation.
	Two-electron integral transformation complete.
	(OO|OV)...
	Starting second half-transformation.
	Two-electron integral transformation complete.
	(OO|VV)...
	Starting second half-transformation.
	Two-electron integral transformation complete.
	(OV|OO)...
	Starting first half-transformation.
	Sorting half-transformed integrals.
	First half integral transformation complete.
	Starting second half-transformation.
	Two-electron integral transformation complete.
	(OV|OV)...
	Starting second half-transformation.
	Two-electron integral transformation complete.
	(OV|VV)...
	Starting second half-transformation.
	Two-electron integral transformation complete.
	(VV|OO)...
	Starting first half-transformation.
	Sorting half-transformed integrals.
	First half integral transformation complete.
	Starting second half-transformation.
	Two-electron integral transformation complete.
	(V

    (T) energy                                =   -0.008502587452393
      * CCSD(T) total energy                  = -257.605070474438548



## QCSchema

* MolSSI format for Molecule, BasisSet, QC input and output
* defined at https://molssi-qc-schema.readthedocs.io/en/latest/index.html
* reference impl at https://github.com/MolSSI/QCElemental/tree/master/qcelemental/models
* JSON, MessagePack, Python dict, QCSchema `AtomicInput`
* can't do everything (e.g., not CC amplitudes return, autofragmentation, etc.)
* run as:
  * `> psi4 --qcschema qcschema.json`
  * in python session, `import psi4`, `psi4.driver.schema_wrapper.run_qcschema(...)`
  * in python session, `import qcengine as qcng`, `qcng.compute(...)`

In [7]:
with open("qcschema.json", "w") as fp:
    fp.write('''{
 "molecule": {
  "symbols": ["Ne", "Ne"],    
  "geometry":
    [0, 0, 0, 5.67, 0, 0]
 },
 "driver": "energy",
 "model": {
  "method": "ccsd(t)",
  "basis": "cc-pvtz"
 },
 "keywords": 
  {"freeze_core": "True"}
}
''')

In [8]:
!psi4 --qcschema qcschema.json
with open("qcschema.json") as fp:
    pprint.pprint(json.load(fp))

{'driver': 'energy',
 'error': None,
 'extras': {'qcvars': {'(T) CORRECTION ENERGY': -0.008502572044481458,
                       'CC D1 DIAGNOSTIC': 0.007944513495760709,
                       'CC D2 DIAGNOSTIC': 0.0861829004588327,
                       'CC NEW D1 DIAGNOSTIC': 0.007944513495760709,
                       'CC T1 DIAGNOSTIC': 0.004506067695692405,
                       'CCSD CORRELATION ENERGY': -0.5328040054919352,
                       'CCSD DOUBLES ENERGY': -0.5328040054919352,
                       'CCSD ITERATIONS': 9.0,
                       'CCSD OPPOSITE-SPIN CORRELATION ENERGY': -0.40499986213804795,
                       'CCSD SAME-SPIN CORRELATION ENERGY': -0.12780414335388723,
                       'CCSD SINGLES ENERGY': 0.0,
                       'CCSD TOTAL ENERGY': -257.5965678841586,
                       'CCSD(T) CORRELATION ENERGY': -0.5413065775364166,
                       'CCSD(T) TOTAL ENERGY': -257.6050704562031,
                     

           '\tTwo-electron integral transformation complete.\n'
           '\t(OV|OO)...\n'
           '\tStarting first half-transformation.\n'
           '\tSorting half-transformed integrals.\n'
           '\tFirst half integral transformation complete.\n'
           '\tStarting second half-transformation.\n'
           '\tTwo-electron integral transformation complete.\n'
           '\t(OV|OV)...\n'
           '\tStarting second half-transformation.\n'
           '\tTwo-electron integral transformation complete.\n'
           '\t(OV|VV)...\n'
           '\tStarting second half-transformation.\n'
           '\tTwo-electron integral transformation complete.\n'
           '\t(VV|OO)...\n'
           '\tStarting first half-transformation.\n'
           '\tSorting half-transformed integrals.\n'
           '\tFirst half integral transformation complete.\n'
           '\tStarting second half-transformation.\n'
           '\tTwo-electron integral transformation complete.\n'
           '\t(V

## Molecules

#### Psithon and PsiAPI use `psi4.core.Molecule`
* supports Cartesian & ZMatrix & deferred, Bohr & Angstrom

#### QCSchema uses `qcelemental.models.Molecule`
* supports only Cartesian in Bohr (but creation is flexible)

#### Creating Molecule from string
* Psithon: `molecule ne {\n 0 3\n Ne 0 0 0\n symmetry c1}`
* PsiAPI: `ne = psi4.geometry("""0 3\n Ne 0 0 0\n symmetry c1""")`
* QCSchema: `ne = qcel.models.Molecule.from_data("""0 3\n Ne 0 0 0\n symmetry c1""")`

#### Creating Molecule from data
* Psithon & PsiAPI

In [9]:
psi4_ne = psi4.core.Molecule.from_arrays(geom=[0, 0, 0], elem=["Ne"], molecular_multiplicity=3, fix_symmetry='c1')
print(psi4_ne)

* QCSchema (notice different kwarg names)

In [10]:
qcsk_ne = qcel.models.Molecule(geometry=[0, 0, 0], symbols=["Ne"], molecular_multiplicity=3, fix_symmetry="c1")
print(qcsk_ne)

Molecule(name='Ne', formula='Ne', hash='a66da3e')


#### Interconverting Molecules
* psi4.core.Molecule <--> qcel.models.Molecule

In [11]:
qcsk_mol = qcel.models.Molecule(**psi4_ne.to_schema(dtype=2))
print(qcsk_mol)

Molecule(name='Ne', formula='Ne', hash='a66da3e')


In [12]:
psi4_mol = psi4.core.Molecule.from_schema(qcsk_ne.dict())
print(psi4_mol)

#### Data from Molecules
* geometry in Bohr

In [13]:
psi4_mol.geometry().np

array([[0., 0., 0.]])

In [14]:
qcsk_mol.geometry

array([[0., 0., 0.]])

* masses and symbols

In [15]:
print([psi4_mol.mass(iat) for iat in range(psi4_mol.natom())])
print([psi4_mol.symbol(iat) for iat in range(psi4_mol.natom())])
# -or-
dmol = psi4_mol.to_dict()
print(dmol["mass"])
print(dmol["elem"])

[19.9924401762]
['NE']
[19.9924401762]
['Ne']


In [16]:
print(qcsk_mol.masses)
print(qcsk_mol.symbols)

[19.9924401762]
['Ne']


## QCVariables
* carefully named pieces of results
* formerly PSI Variables
* now for arrays as well as scalars
* usually atomic units
* in globals (boo) and on wavefunction (yay)

#### QCVariables from Psithon and PsiAPI

In [17]:
psi4.set_options({"reference": "uhf"})
e, wfn = psi4.energy("ccsd/cc-pvdz", molecule=psi4_mol, return_wfn=True)
psi4.core.variables() 

{'CC D1 DIAGNOSTIC': 0.0,
 'CC D2 DIAGNOSTIC': 0.0,
 'CC NEW D1 DIAGNOSTIC': 0.0,
 'CC T1 DIAGNOSTIC': 0.00522595952002172,
 'CCSD CORRELATION ENERGY': -0.20273659019175833,
 'CCSD DOUBLES ENERGY': -0.20273659019175833,
 'CCSD ITERATIONS': 13.0,
 'CCSD OPPOSITE-SPIN CORRELATION ENERGY': -0.1724019571297366,
 'CCSD SAME-SPIN CORRELATION ENERGY': -0.030334633062021728,
 'CCSD SINGLES ENERGY': 6.0316867478082724e-18,
 'CCSD TOTAL ENERGY': -127.0059578014308,
 'CURRENT CORRELATION ENERGY': -0.20273659019175833,
 'CURRENT DIPOLE X': 3.7065764627832447e-16,
 'CURRENT DIPOLE Y': 1.2306873303502418e-15,
 'CURRENT DIPOLE Z': -8.418989624900764e-17,
 'CURRENT ENERGY': -127.0059578014308,
 'CURRENT REFERENCE ENERGY': -126.80322121123905,
 'HF TOTAL ENERGY': -126.80322121123905,
 'LCCSD (+LMP2) TOTAL ENERGY': -127.0059578014308,
 'MP2 CORRELATION ENERGY': -0.13589138945099172,
 'MP2 DOUBLES ENERGY': -0.13589138945099172,
 'MP2 OPPOSITE-SPIN CORRELATION ENERGY': -0.10462807025215873,
 'MP2 SAME-SPI

In [18]:
# * wfn newer and more reliable than globals
# * lists may not be identical
print(f"{wfn.variable('CCSD TOTAL ENERGY')=}")
wfn.variables()

wfn.variable('CCSD TOTAL ENERGY')=-127.0059578014308


{'CCSD CORRELATION ENERGY': -0.20273659019175833,
 'CCSD DOUBLES ENERGY': -0.20273659019175833,
 'CCSD ITERATIONS': 13.0,
 'CCSD OPPOSITE-SPIN CORRELATION ENERGY': -0.1724019571297366,
 'CCSD SAME-SPIN CORRELATION ENERGY': -0.030334633062021728,
 'CCSD SINGLES ENERGY': 6.0316867478082724e-18,
 'CCSD TOTAL ENERGY': -127.0059578014308,
 'CURRENT CORRELATION ENERGY': -0.20273659019175833,
 'CURRENT DIPOLE X': 3.7065764627832447e-16,
 'CURRENT DIPOLE Y': 1.2306873303502418e-15,
 'CURRENT DIPOLE Z': -8.418989624900764e-17,
 'CURRENT ENERGY': -127.0059578014308,
 'CURRENT REFERENCE ENERGY': -126.80322121123905,
 'HF TOTAL ENERGY': -126.80322121123905,
 'MP2 CORRELATION ENERGY': -0.13589138945099172,
 'MP2 DOUBLES ENERGY': -0.13589138945099172,
 'MP2 OPPOSITE-SPIN CORRELATION ENERGY': -0.10462807025215873,
 'MP2 SAME-SPIN CORRELATION ENERGY': -0.031263319198833,
 'MP2 SINGLES ENERGY': -2.682310940717586e-30,
 'MP2 TOTAL ENERGY': -126.93911260069002,
 'NUCLEAR REPULSION ENERGY': 0.0,
 'ONE-ELE

#### Use QCElemental for unit conversion

In [19]:
import numpy as np
au2D = 2.541746451895025916414946904
au2Q = au2D * 0.52917721067

onebuckingham = qcel.Datum("CC quadrupole", "e a0^2", np.array([0, 0, 1 / au2Q, 0, 0, 0, 0, 0, 0]).reshape((3, 3)))

onebuckingham.to_units("D Å")

array([[0.                , 0.                , 0.9999999997005935],
       [0.                , 0.                , 0.                ],
       [0.                , 0.                , 0.                ]])

#### QCVariables from QCSchema
* exact duplicates stashed in "extras"
* many translated to QCSchema properties

In [20]:
atomicinput = qcel.models.AtomicInput(**{"molecule": qcsk_mol, "driver": "energy", "model":{"method": "ccsd", "basis": "cc-pvdz"}, "keywords": {"reference": "uhf"}})
pprint.pprint(atomicinput.dict())

{'driver': <DriverEnum.energy: 'energy'>,
 'extras': {},
 'id': None,
 'keywords': {'reference': 'uhf'},
 'model': {'basis': 'cc-pvdz', 'method': 'ccsd'},
 'molecule': {'atom_labels': array([''], dtype='<U1'),
              'atomic_numbers': array([10], dtype=int16),
              'fix_com': False,
              'fix_orientation': False,
              'fix_symmetry': 'c1',
              'fragment_charges': [0.0],
              'fragment_multiplicities': [3],
              'fragments': [array([0], dtype=int32)],
              'geometry': array([[0., 0., 0.]]),
              'mass_numbers': array([20], dtype=int16),
              'masses': array([19.9924401762]),
              'molecular_charge': 0.0,
              'molecular_multiplicity': 3,
              'name': 'Ne',
              'provenance': {'creator': 'QCElemental',
                             'routine': 'qcelemental.molparse.from_arrays',
                             'version': 'v0.16.0'},
              'real': array([ True]),

In [21]:
atomicresult = qcng.compute(atomicinput, "psi4", local_options={"memory": 0.5, "ncores": 2})
pprint.pprint(atomicresult.dict())

{'driver': <DriverEnum.energy: 'energy'>,
 'error': None,
 'extras': {'qcvars': {'CC D1 DIAGNOSTIC': 0.0,
                       'CC D2 DIAGNOSTIC': 0.0,
                       'CC NEW D1 DIAGNOSTIC': 0.0,
                       'CC T1 DIAGNOSTIC': 0.005225959520021711,
                       'CCSD CORRELATION ENERGY': -0.20273659019175816,
                       'CCSD DOUBLES ENERGY': -0.20273659019175816,
                       'CCSD ITERATIONS': 13.0,
                       'CCSD OPPOSITE-SPIN CORRELATION ENERGY': -0.17240195712973644,
                       'CCSD SAME-SPIN CORRELATION ENERGY': -0.030334633062021735,
                       'CCSD SINGLES ENERGY': 6.031686747808264e-18,
                       'CCSD TOTAL ENERGY': -127.0059578014308,
                       'CURRENT CORRELATION ENERGY': -0.20273659019175816,
                       'CURRENT DIPOLE': array([ 1.4582793889688592e-16,  4.8418965213178117e-16,
       -3.3122853849658752e-17]),
                       'CURRENT 

In [22]:
print(f"{atomicresult.extras['qcvars']['CCSD TOTAL ENERGY']=}")
print(f"{atomicresult.properties.ccsd_total_energy=}")

atomicresult.extras['qcvars']['CCSD TOTAL ENERGY']=-127.0059578014308
atomicresult.properties.ccsd_total_energy=-127.0059578014308


## Wavefunction Results
* well developed in Psi4 (https://github.com/psi4/psi4numpy/blob/master/Tutorials/01_Psi4NumPy-Basics/1d_wavefunction.ipynb)
* SCF only at present in QCSchema

In [23]:
wfn.epsilon_a().np

array([-32.918380569165855 ,  -2.132995100493552 ,  -1.2292764705485981,
        -0.9905964869109233,  -0.9905964869109204,   0.9332539325327035,
         1.582893110221902 ,   1.582893110221907 ,   1.9506131626694976,
         4.962615893615121 ,   4.984041064309092 ,   4.984041064309095 ,
         5.053333421212122 ,   5.053333421212125 ])

In [24]:
wfn.Ca().np

array([[ 1.0012501709247628e+00,  1.3385161764875184e-02,
         3.2915753674940459e-17,  8.3374227971766224e-18,
         1.8077138066450155e-17, -2.6652334885396105e-16,
         7.4489879157477140e-17, -2.2373839418830838e-16,
         6.1999441173463488e-01,  1.0074157938918731e-02,
        -2.5897228606711071e-16, -6.4799823501283319e-17,
        -1.0309647502521734e-16, -1.5433549725656749e-16],
       [ 2.7933526902245158e-03,  5.5433531316803697e-01,
        -1.2852201627427876e-18, -1.5171275138541860e-17,
        -1.2300723140630388e-18, -6.4093370366023696e-16,
         1.5352514523781976e-16, -5.1067297679123612e-16,
         1.5296249666099424e+00,  2.9628147383101159e-02,
        -8.6114474614121170e-16, -1.0563085393821374e-16,
        -4.2661683416234242e-16, -4.2484958956385455e-16],
       [-3.3961692645530922e-03,  5.2885199331661381e-01,
        -1.4667680227886918e-16, -6.5622681972916933e-17,
        -1.6657453230070897e-16,  6.4476964036961120e-16,
        -2.2

In [25]:
atomicinput = atomicinput.dict()  # immutable
atomicinput["protocols"]["wavefunction"] = "orbitals_and_eigenvalues"
pprint.pprint(atomicinput)

{'driver': <DriverEnum.energy: 'energy'>,
 'extras': {},
 'id': None,
 'keywords': {'reference': 'uhf'},
 'model': {'basis': 'cc-pvdz', 'method': 'ccsd'},
 'molecule': {'atom_labels': array([''], dtype='<U1'),
              'atomic_numbers': array([10], dtype=int16),
              'fix_com': False,
              'fix_orientation': False,
              'fix_symmetry': 'c1',
              'fragment_charges': [0.0],
              'fragment_multiplicities': [3],
              'fragments': [array([0], dtype=int32)],
              'geometry': array([[0., 0., 0.]]),
              'mass_numbers': array([20], dtype=int16),
              'masses': array([19.9924401762]),
              'molecular_charge': 0.0,
              'molecular_multiplicity': 3,
              'name': 'Ne',
              'provenance': {'creator': 'QCElemental',
                             'routine': 'qcelemental.molparse.from_arrays',
                             'version': 'v0.16.0'},
              'real': array([ True]),

In [26]:
atomicresult = qcng.compute(atomicinput, "psi4", local_options={"memory": 0.5, "ncores": 2})
#pprint.pprint(atomicresult.dict())
print(atomicresult.wavefunction.scf_eigenvalues_a)
print(atomicresult.wavefunction.scf_orbitals_a)  # transformed to CCA

[-32.918380569165855   -2.132995100493552   -1.2292764705485981
  -0.9905964869109204  -0.9905964869109233   0.9332539325327035
   1.582893110221907    1.582893110221902    1.9506131626694976
   5.053333421212122    4.984041064309092    4.962615893615121
   4.984041064309095    5.053333421212125 ]
[[ 1.0012501709247628e+00  1.3385161764875184e-02  3.2915753674940459e-17
   8.3374227971766224e-18  1.8077138066450155e-17 -2.6652334885396105e-16
   7.4489879157477140e-17 -2.2373839418830838e-16  6.1999441173463488e-01
   1.0074157938918731e-02 -2.5897228606711071e-16 -6.4799823501283319e-17
  -1.0309647502521734e-16 -1.5433549725656749e-16]
 [ 2.7933526902245158e-03  5.5433531316803697e-01 -1.2852201627427876e-18
  -1.5171275138541860e-17 -1.2300723140630388e-18 -6.4093370366023696e-16
   1.5352514523781976e-16 -5.1067297679123612e-16  1.5296249666099424e+00
   2.9628147383101159e-02 -8.6114474614121170e-16 -1.0563085393821374e-16
  -4.2661683416234242e-16 -4.2484958956385455e-16]
 [-3.39

# When to use each input mode

### Psithon
* new user who doesn't know Python syntax and who will never need to
* users running individual, independent jobs and writing input by hand
* tests found at https://github.com/psi4/psi4/tree/master/tests

### QCSchema
* machines formulating, running, and harvesting jobs
* users running non-interactive and independent jobs through Python
* tests found at https://github.com/psi4/psi4/tree/master/tests/json

### PsiAPI
* everyone else
* at the cost of a few more quotes and `psi4.`s, avoid the DSL
* tests found at https://github.com/psi4/psi4/tree/master/tests/pytests

In [27]:
# to keynote